In [ ]:
import spyker, torch
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
batch, root = 64, './MNIST'
trainx, trainy, testx, testy = spyker.read_mnist(
    root+'/train-images-idx3-ubyte', root+'/train-labels-idx1-ubyte',
    root+'/t10k-images-idx3-ubyte', root+'/t10k-labels-idx1-ubyte')
trainx, trainy, testx, testy = spyker.to_torch(trainx, trainy, testx, testy)
train = DataLoader(TensorDataset(trainx, trainy), batch_size=batch)
test = DataLoader(TensorDataset(testx, testy), batch_size=batch)

In [ ]:
class Network:
    def __init__(self, device):
        self.device = device
        self.filter = spyker.LoG(3, [.5, 1, 2], pad=3, device=device)
        self.conv1 = spyker.Conv(6, 50, 5, pad=2, device=device)
        self.conv2 = spyker.Conv(50, 100, 3, pad=1, device=device)
        self.conv1.stdpconfig = [spyker.STDPConfig(.004, -.003)]
        self.conv2.stdpconfig = [spyker.STDPConfig(.004, -.003)]
    
    def code(self, input, code):
        if self.device.kind == 'cuda': input = input.cuda()
        return spyker.code(spyker.threshold(self.filter(input), .01), 15, code=code)
    
    def train1(self, input):
        input = self.code(input, 'rank')
        output = spyker.inhibit(spyker.threshold(self.conv1(input), 16))
        self.conv1.stdp(input, spyker.convwta(output, 3, 5), spyker.fire(output))
        
    def train2(self, input):
        input = self.code(input, 'rank')
        input = spyker.pool(spyker.fire(self.conv1(input), 16), 2)
        output = spyker.inhibit(spyker.threshold(self.conv2(input), 5))
        self.conv2.stdp(input, spyker.convwta(output, 1, 8), spyker.fire(output))
    
    def __call__(self, input):
        input = self.code(input, 'rate')
        input = spyker.fire(self.conv1(input), 16, code='rate')
        input = spyker.pool(input, 2, rates=spyker.gather(input, code='rate'))
        input = spyker.fire(self.conv2(input), 5, code='rate')
        input = spyker.pool(input, 3, rates=spyker.gather(input, code='rate'))
        return spyker.gather(input, code='rate').flatten(1)

In [ ]:
def Total(network, dataset):
    data_total, target_total = [], []
    for data, target in dataset:
        data_total.append(network(data).cpu())
        target_total.append(target)
    return torch.cat(data_total), torch.cat(target_total)

In [ ]:
def Update(config):
    rate = config.negative / config.positive
    pos = min(config.positive * 2, .1)
    config.positive, config.negative = pos, pos * rate

In [ ]:
device = spyker.device('cuda' if spyker.cuda_available() else 'cpu')

In [ ]:
network = Network(device)

In [ ]:
from tqdm.notebook import tqdm

for i, (data, _) in enumerate(tqdm(train, "Training Layer 1")):
    if (i + 1) % 10 == 0: Update(network.conv1.stdpconfig[0])
    network.train1(data)
spyker.quantize(network.conv1.kernel, 0, .5, 1)

for i, (data, _) in enumerate(tqdm(train, "Training Layer 2")):
    if (i + 1) % 10 == 0: Update(network.conv2.stdpconfig[0])
    network.train2(data)
spyker.quantize(network.conv2.kernel, 0, .5, 1);

In [ ]:
train_data, train_target = Total(network, train)
test_data, test_target = Total(network, test)

In [ ]:
pca = PCA(n_components=200).fit(train_data, train_target)
train_data, test_data = pca.transform(train_data), pca.transform(test_data)

In [ ]:
target = SVC(C=2.4).fit(train_data, train_target).predict(test_data)
accuracy = (torch.tensor(target) == test_target).sum() / len(test_target)
print(f"Final Accuracy: {accuracy * 100 :.2f}%")